In [ ]:
import os
import sys
from google.colab.patches import cv2_imshow
from tensorflow.keras.applications import InceptionV3
import random
import numpy as np
import cv2
import tensorflow as tf
from PIL import Image, ImageDraw, ImageFont
import random

# Local path -- This adds the faceRecognition module
sys.path.append(os.path.join('.', 'drive', 'My Drive', 'IZ*Net', 'faceRecognition'))
from analysis import Analysis
from consts import BASE_PATH, TRAIN_PATH, FACES_PATH
from util import load_img
from models.nn2 import inception_network

In [ ]:
# Copy-pasted from FaceDetection.ipynb -- Needed to define the DarkNet-53 architecture
from keras import Model
from keras.layers import Conv2D, Dense, Flatten, Reshape, Input, BatchNormalization, MaxPooling2D, LeakyReLU, Add

def Conv(XInput, filters, kernel_size, strides):
  X = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same')(XInput)
  X = BatchNormalization()(X)
  X = LeakyReLU(0.1)(X)
  return X

def Residual(XInput, filters):
  XBranch = Conv(XInput, filters=filters, kernel_size=1, strides=1)
  XBranch = Conv(XBranch, filters=filters * 2, kernel_size=3, strides=1)
  X = Add()([XInput, XBranch])
  return X

def network():
  X_input = Input((416, 416, 3))

  X = Conv(X_input, filters=32, kernel_size=3, strides=1)
  X = Conv(X, filters=64, kernel_size=3, strides=2)
  X = Residual(X, filters=32)
  X = Conv(X, filters=128, kernel_size=3, strides=2)
  X = Residual(X, filters=64)
  X = Residual(X, filters=64)
  X = Conv(X, filters=256, kernel_size=3, strides=2)
  X = Residual(X, filters=128)
  X = Residual(X, filters=128)
  X = Residual(X, filters=128)
  X = Residual(X, filters=128)
  X = Residual(X, filters=128)
  X = Residual(X, filters=128)
  X = Residual(X, filters=128)
  X = Residual(X, filters=128)
  X = Conv(X, filters=512, kernel_size=3, strides=2)
  X = Residual(X, filters=256)
  X = Residual(X, filters=256)
  X = Residual(X, filters=256)
  X = Residual(X, filters=256)
  X = Residual(X, filters=256)
  X = Residual(X, filters=256)
  X = Residual(X, filters=256)
  X = Residual(X, filters=256)
  X = Conv(X, filters=1024, kernel_size=3, strides=2)
  X = Residual(X, filters=512)
  X = Residual(X, filters=512)
  X = Residual(X, filters=512)
  X = Residual(X, filters=512)
  X = Conv2D(128, 1)(X)
  X = Flatten()(X)
  X = Dense(1690)(X)
  X = Reshape((13, 13, 2, 5))(X)

  return Model(inputs=X_input, outputs=X)

In [ ]:
# Load the models with the trained weights
faceDetectionModel = network()
faceDetectionModel.load_weights(os.path.join(BASE_PATH, 'network4_yolo_highestMeanAccuracy.h5'))

faceRecoModelWeights = os.path.join(BASE_PATH, 'nn2_binary_304640.h5')
faceRecoModel = inception_network()
faceRecoModel.load_weights(faceRecoModelWeights)

analysis = Analysis(faceRecoModel)
analysis.visualize_medoids_table('test')
MEDOIDS = {member: analysis.embeddings[member][index] for (member, index) in analysis.medoids_index.items()}
MEDOIDS = {member: embedding/np.linalg.norm(embedding) for (member, embedding) in MEDOIDS.items()}

In [ ]:
# Define a couple more auxiliary functions

ANCHOR_SIZES = np.array([[1, 1], [1, 1.15]])

def load_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (416, 416))
    img = img[..., ::-1]  # Reverse channels
    img = np.around(img/255.0, decimals=12)  # Normalize
    return img

def getBoundingBoxCoordinates(boundingBoxRow, boundingBoxCol, anchorBox, boundingBoxInfo, imageWidth, imageHeight):
    widthScale = imageWidth / 13
    heightScale = imageHeight / 13

    (bbInfoX, bbInfoY) = tf.sigmoid(boundingBoxInfo[:2])
    (bbInfoW, bbInfoH) = tf.exp(boundingBoxInfo[2:]) * ANCHOR_SIZES[anchorBox]
    
    midX = bbInfoX + boundingBoxCol
    midY = bbInfoY + boundingBoxRow

    startX = (midX - (bbInfoW / 2.)) * widthScale
    startY = (midY - (bbInfoH / 2.)) * heightScale
    endX = (midX + (bbInfoW / 2.)) * widthScale
    endY = (midY + (bbInfoH / 2.)) * heightScale

    return [startY, startX, endY, endX]

def getBoundingBoxes(path, imageWidth, imageHeight):
  image = load_img(path)
  faceDetectionOutput = tf.reshape(faceDetectionModel(np.array([image])), (13, 13, 2 ,5))

  boundingBoxes = []
  confidenceScores = []
  for row in range(13):
      for col in range(13):
        for boxNum in range(2):
          boundingBoxes.append(getBoundingBoxCoordinates(row, col, boxNum, faceDetectionOutput[row][col][boxNum][1:], imageWidth, imageHeight))
          confidenceScores.append(tf.sigmoid(faceDetectionOutput[row][col][boxNum][0]))

  selectedIndices = tf.image.non_max_suppression(boundingBoxes, confidenceScores, max_output_size=12, iou_threshold=0.4)
  selectedBoxes = tf.gather(boundingBoxes, selectedIndices)
  selectedBoxesConfidences = tf.gather(confidenceScores, selectedIndices)

  return (selectedBoxes, selectedBoxesConfidences)

def getClosestMember(image):
  embedding = np.squeeze(faceRecoModel(image))
  embedding = embedding/np.linalg.norm(embedding)
  minDist = 999999999
  minMember = None

  for member, memberEmbedding in MEDOIDS.items():
    dist = np.linalg.norm(embedding - memberEmbedding)
    if dist < minDist:
      minDist = dist
      minMember = member

  return minMember if minDist <= 1.5 else None

def drawBox(image, box, member):
  Font = ImageFont.truetype(os.path.join(BASE_PATH, 'Roboto-Regular.ttf'), 20)

  width, height = image.size

  Draw = ImageDraw.Draw(image)
  labelSize = Draw.textsize(member, Font)

  [startY, startX, endY, endX] = box

  for i in range(2):
    Draw.rectangle([(startX + i, startY + i), (endX - i, endY - i)], outline = 'red')

  if startY - labelSize[1] >= 0:
      textOrigin = np.array([startX, startY - labelSize[1]])
  else:
      textOrigin = np.array([startX, startY + 1])
  
  Draw.rectangle([tuple(textOrigin), tuple(textOrigin + labelSize)], fill='red')
  Draw.text(textOrigin, member, fill=(0, 0, 0), font=Font)

def drawImage(imagePath):
  theImage = Image.open(imagePath).convert('RGB')
  imageWidth, imageHeight = theImage.size
  theImage = theImage.resize((imageWidth // 5, imageHeight // 5))

  (boundingBoxes, confidences) = getBoundingBoxes(imagePath, imageWidth // 5, imageHeight // 5)

  members = []
  boxes = []
  for (boundingBox, confidence) in zip(boundingBoxes, confidences):
    if confidence < 0.3: 
      continue
  
    [startY, startX, endY, endX] = [int(scalar) for scalar in boundingBox]
    memberCrop = theImage.crop((startX, startY, endX, endY))
    memberCrop = memberCrop.resize((224, 224))
    memberCrop = np.array(memberCrop)[...,::-1]
    memberCrop = np.around(memberCrop/255.0, decimals=12)
    memberCrop = np.expand_dims(memberCrop, axis=0)
    member = getClosestMember(memberCrop)

    if member is not None:
      members.append(member)
      boxes.append(boundingBox)

  for (member, box) in zip(members, boxes):
    drawBox(theImage, box, member)
  
  cv2_imshow(np.array(theImage)[...,::-1])

In [ ]:
# Local paths to images
paths = [os.path.join(BASE_PATH, 'train', member, img) for member in os.listdir(os.path.join(BASE_PATH, 'train')) for img in os.listdir(os.path.join(BASE_PATH, 'train', member))]

In [ ]:
# Draw!
randomImgs = random.sample(paths, k=5)
for img in randomImgs:
  drawImage(img)